In [163]:
import pandas as pd
from sodapy import Socrata
from uszipcode import SearchEngine # note this does an approximation of zipcode based upon a radius
from datetime import datetime
from datetime import timedelta

import pickle

In [164]:
pd.options.display.max_columns=100

In [187]:
client = Socrata("data.lacity.org", None)

# results = client.get("7fvc-faax", limit=2000000)
results = client.get("7fvc-faax", limit=10000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [188]:
# results_df.head()

In [189]:
# with open('results_df.pickle', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(results_df, f, pickle.HIGHEST_PROTOCOL)

In [190]:
# with open('results_df.pickle', 'rb') as f:
#     # The protocol version used is detected automatically, so we do not
#     # have to specify it.
#     results_df = pickle.load(f)

In [191]:
list(results_df)

[':@computed_region_2dna_qi2s',
 ':@computed_region_k96s_3jcv',
 ':@computed_region_kqwf_mjcx',
 ':@computed_region_qz3q_ghft',
 ':@computed_region_tatf_ua23',
 ':@computed_region_ur2y_g4cx',
 'area_id',
 'area_name',
 'crm_cd',
 'crm_cd_1',
 'crm_cd_2',
 'crm_cd_3',
 'crm_cd_4',
 'crm_cd_desc',
 'cross_street',
 'date_occ',
 'date_rptd',
 'dr_no',
 'location',
 'location_1',
 'mocodes',
 'premis_cd',
 'premis_desc',
 'rpt_dist_no',
 'status',
 'status_desc',
 'time_occ',
 'vict_age',
 'vict_descent',
 'vict_sex',
 'weapon_desc',
 'weapon_used_cd']

In [ ]:
rows=len(results_df)
search = SearchEngine(simple_zipcode=True)
lon = []
lat = []
zipcode = []
year_single = '2000'
year = []
for i in range(rows):
    # single values for readability
    lattitude = float(results_df['location_1'][i]['coordinates'][1])
    longitude = float(results_df['location_1'][i]['coordinates'][0])
    year_single = results_df['date_occ'][i][:4]
    # append to arrays
    print(f'{i},{lattitude},{longitude}')
    lon.append(longitude)
    lat.append(lattitude)
    try:
        zipcode.append(search.by_coordinates(lattitude, longitude, radius=100, returns=1)[0].zipcode)
    except:
        zipcode.append('000000')
    year.append(year_single)
results_df['lat'] = lat
results_df['lon'] = lon
results_df['zipcode'] = zipcode
results_df['year'] = year

0,34.0761,-118.3555
1,34.0649,-118.3451
2,34.073,-118.3765
3,34.0516,-118.3219
4,34.0399,-118.3353
5,34.0399,-118.3353
6,34.0623,-118.3462
7,34.0485,-118.3558
8,34.0421,-118.3456
9,34.0481,-118.3504
10,34.0516,-118.3219
11,34.0377,-118.3351
12,34.0355,-118.3508
13,34.0835,-118.3416
14,34.0461,-118.3441
15,34.0399,-118.3435
16,34.0399,-118.3398
17,34.0459,-118.3374
18,34.0479,-118.3447
19,34.0465,-118.351
20,34.0399,-118.3311
21,34.0418,-118.3269
22,34.0842,-118.3765
23,34.0776,-118.3273
24,34.0459,-118.3454
25,34.0344,-118.3497
26,34.0649,-118.3525
27,34.0463,-118.3387
28,34.0531,-118.3238
29,34.0721,-118.3857
30,34.0649,-118.3472
31,34.0399,-118.3248
32,34.0541,-118.3547
33,34.0412,-118.3317
34,34.0537,-118.3288
35,34.0399,-118.326
36,34.0455,-118.3177
37,34.0414,-118.3392
38,34.0399,-118.3324
39,34.0466,-118.3414
40,34.0436,-118.34
41,34.0782,-118.3594
42,34.0375,-118.3742
43,34.0375,-118.3742
44,34.0371,-118.3236
45,34.0446,-118.3526
46,34.0623,-118.3462
47,34.0446,-118.3526
48,34.0

380,34.043,-118.242
381,34.0473,-118.2462
382,34.0607,-118.2494
383,34.0509,-118.431
384,34.0406,-118.4275
385,34.0625,-118.444
386,34.05,-118.5479
387,34.0382,-118.2665
388,34.0472,-118.2371
389,34.0495,-118.2529
390,34.0683,-118.245
391,34.0594,-118.3803
392,34.0625,-118.447
393,34.0548,-118.3837
394,34.0354,-118.446
395,34.0594,-118.3824
396,34.0419,-118.4602
397,34.2012,-118.4662
398,34.164,-118.4662
399,34.0488,-118.2553
400,34.1523,-118.4641
401,34.153,-118.4487
402,34.0357,-118.2563
403,34.2085,-118.4618
404,34.174,-118.4662
405,34.1506,-118.4411
406,34.1951,-118.4241
407,34.2085,-118.4662
408,34.0467,-118.2556
409,34.0724,-118.2486
410,34.0736,-118.2312
411,34.0467,-118.2485
412,34.1887,-118.4531
413,34.1687,-118.4153
414,34.1612,-118.4574
415,34.1463,-118.4523
416,34.203,-118.5192
417,34.0502,-118.254
418,34.1615,-118.5142
419,34.1889,-118.4924
420,34.2175,-118.5361
421,34.193,-118.508
422,33.9454,-118.3998
423,34.0317,-118.2641
424,34.1977,-118.5415
425,34.1018,-118.283
426,3

760,34.103,-118.3225
761,34.1041,-118.3417
762,34.1006,-118.3417
763,34.0953,-118.3223
764,34.1018,-118.3072
765,34.1016,-118.3452
766,34.1113,-118.3335
767,34.1333,-118.3438
768,34.1053,-118.3724
769,34.1016,-118.331
770,34.104,-118.3448
771,34.1034,-118.3325
772,34.098,-118.3338
773,34.0998,-118.3267
774,34.0944,-118.355
775,34.1016,-118.331
776,33.7415,-118.2857
777,34.1847,-118.4509
778,34.1036,-118.3049
779,34.0981,-118.3615
780,34.0908,-118.3201
781,34.0908,-118.3201
782,34.0889,-118.3321
783,34.1034,-118.3325
784,34.098,-118.3656
785,34.103,-118.3402
786,34.1017,-118.355
787,34.1026,-118.3123
788,34.0865,-118.3169
789,34.1084,-118.3213
790,34.1053,-118.3224
791,34.1016,-118.3433
792,34.0998,-118.3211
793,34.0994,-118.3093
794,34.0967,-118.3056
795,34.1036,-118.3049
796,34.0907,-118.3363
797,34.1016,-118.3417
798,34.1016,-118.3417
799,34.103,-118.318
800,34.1039,-118.3311
801,34.0926,-118.3374
802,34.1036,-118.3049
803,34.0938,-118.3109
804,34.1017,-118.318
805,34.0962,-118.3425


1133,33.9712,-118.2608
1134,33.9829,-118.3338
1135,33.9963,-118.3149
1136,33.9624,-118.2761
1137,33.9628,-118.294
1138,33.9661,-118.2849
1139,33.9655,-118.2893
1140,33.9821,-118.3318
1141,33.9708,-118.2783
1142,33.9745,-118.3309
1143,33.9673,-118.3003
1144,33.9872,-118.3378
1145,33.9693,-118.2652
1146,33.9965,-118.2827
1147,33.9859,-118.3002
1148,33.9837,-118.3206
1149,33.9655,-118.2871
1150,34.0017,-118.2896
1151,33.7735,-118.2737
1152,33.9622,-118.285
1153,33.9655,-118.3025
1154,33.9789,-118.2981
1155,33.9619,-118.2695
1156,33.9746,-118.3294
1157,33.96,-118.3003
1158,33.9843,-118.2915
1159,33.9863,-118.2881
1160,33.9826,-118.3529
1161,33.979,-118.3112
1162,33.9719,-118.3003
1163,33.9655,-118.3025
1164,33.9661,-118.2849
1165,33.9588,-118.3131
1166,33.9588,-118.3131
1167,33.9588,-118.3131
1168,33.9588,-118.3131
1169,33.9845,-118.3352
1170,34.0002,-118.2499
1171,34.0039,-118.2392
1172,33.9953,-118.2701
1173,34.0096,-118.244
1174,34.0312,-118.2478
1175,33.9983,-118.2783
1176,34.0357,-118

1494,34.3154,-118.4227
1495,34.2492,-118.4452
1496,34.2355,-118.4632
1497,34.3214,-118.4252
1498,34.3214,-118.4252
1499,34.2479,-118.4698
1500,34.2415,-118.4429
1501,34.2573,-118.4674
1502,34.3261,-118.4274
1503,34.3152,-118.4314
1504,34.2231,-118.4677
1505,34.2254,-118.4405
1506,34.2228,-118.4303
1507,34.2537,-118.4355
1508,34.2153,-118.4519
1509,34.2702,-118.4672
1510,34.2355,-118.4676
1511,34.2918,-118.4506
1512,34.2427,-118.4702
1513,34.2427,-118.4702
1514,34.3155,-118.4343
1515,34.2299,-118.4516
1516,34.3192,-118.4455
1517,34.2255,-118.438
1518,34.3221,-118.422
1519,34.2246,-118.4622
1520,34.2265,-118.4565
1521,34.2926,-118.4597
1522,34.2464,-118.4631
1523,34.2269,-118.4394
1524,34.2991,-118.4638
1525,34.0654,-118.2941
1526,34.0578,-118.2916
1527,34.0709,-118.2954
1528,34.0709,-118.2954
1529,34.0527,-118.3091
1530,34.044,-118.2971
1531,34.0509,-118.3
1532,34.0588,-118.2842
1533,34.0672,-118.2966
1534,34.0577,-118.3066
1535,34.0427,-118.309
1536,34.0709,-118.2985
1537,34.0691,-118.

...

3667,34.0151,-118.4936
3668,34.032,-118.2667
3669,34.0598,-118.2477
3670,34.0452,-118.2534
3671,34.0442,-118.2439
3672,34.0578,-118.252
3673,34.0524,-118.2467
3674,34.0446,-118.249
3675,34.0467,-118.2556
3676,34.0535,-118.256
3677,34.0449,-118.2458
3678,34.0435,-118.2471
3679,34.0495,-118.2494
3680,34.0481,-118.2507
3681,34.0637,-118.244
3682,34.048,-118.2577
3683,34.0423,-118.2452
3684,34.0646,-118.2473
3685,34.0423,-118.2452
3686,34.0416,-118.255
3687,34.0408,-118.2428
3688,34.0481,-118.2507
3689,34.0649,-118.2465
3690,34.01,-118.1166
3691,34.043,-118.2571
3692,34.0435,-118.2471
3693,34.0565,-118.2395
3694,34.0498,-118.24
3695,34.0423,-118.2452
3696,34.0494,-118.2581
3697,34.0372,-118.2654
3698,34.0623,-118.2383
3699,34.0495,-118.2494
3700,34.0459,-118.2351
3701,34.0481,-118.2507
3702,34.0418,-118.2351
3703,34.0643,-118.2299
3704,34.048,-118.2577
3705,34.0435,-118.2471
3706,34.048,-118.2577
3707,34.0472,-118.2361
3708,34.0594,-118.2452
3709,34.0357,-118.2563
3710,34.0454,-118.236
371

4029,34.0495,-118.2494
4030,34.0481,-118.2507
4031,34.0485,-118.2447
4032,34.045,-118.264
4033,34.0416,-118.2584
4034,34.0495,-118.2529
4035,34.0487,-118.2571
4036,34.0479,-118.2612
4037,34.045,-118.264
4038,34.0467,-118.2556
4039,34.0439,-118.2479
4040,34.0668,-118.2359
4041,34.0454,-118.2466
4042,34.0388,-118.2574
4043,34.0373,-118.2637
4044,34.0419,-118.2523
4045,34.0445,-118.2523
4046,34.033,-118.2615
4047,34.0472,-118.2371
4048,34.0435,-118.2471
4049,34.045,-118.264
4050,34.0412,-118.2474
4051,34.0489,-118.2453
4052,34.0423,-118.2452
4053,34.0408,-118.2428
4054,34.0452,-118.2534
4055,34.0487,-118.2588
4056,34.0451,-118.2604
4057,34.0549,-118.2426
4058,34.0461,-118.2476
4059,34.0461,-118.2476
4060,34.0669,-118.2458
4061,34.051,-118.2445
4062,34.0689,-118.2459
4063,34.0434,-118.2577
4064,34.0401,-118.2668
4065,34.0438,-118.2547
4066,34.0467,-118.2485
4067,34.051,-118.248
4068,34.0693,-118.2488
4069,34.0363,-118.2573
4070,34.043,-118.242
4071,34.043,-118.2571
4072,34.0428,-118.2461
4

4389,34.0454,-118.2466
4390,34.0358,-118.2334
4391,34.0544,-118.2466
4392,34.0472,-118.2442
4393,34.067,-118.2408
4394,34.0348,-118.2687
4395,34.048,-118.2577
4396,34.0481,-118.2507
4397,34.0626,-118.2433
4398,34.0389,-118.2643
4399,34.046,-118.2509
4400,34.0446,-118.245
4401,34.0473,-118.2566
4402,34.0423,-118.2452
4403,34.0437,-118.2652
4404,34.0439,-118.2514
4405,34.0509,-118.2605
4406,34.0481,-118.2507
4407,34.048,-118.2577
4408,34.0661,-118.2476
4409,34.0502,-118.254
4410,34.0467,-118.2485
4411,34.046,-118.2509
4412,34.0461,-118.2476
4413,34.0423,-118.2452
4414,34.0449,-118.2458
4415,34.0475,-118.2657
4416,34.0461,-118.2476
4417,34.0428,-118.2461
4418,34.0565,-118.2395
4419,34.0661,-118.2334
4420,34.0419,-118.2523
4421,34.0423,-118.2452
4422,34.0455,-118.2381
4423,34.0442,-118.2439
4424,34.0633,-118.229
4425,34.0454,-118.2466
4426,34.0459,-118.2657
4427,34.0455,-118.2381
4428,34.0583,-118.2395
4429,34.0449,-118.2458
4430,34.0688,-118.2291
4431,34.0479,-118.2612
4432,34.0442,-118.2

4748,34.0423,-118.2666
4749,34.0371,-118.2551
4750,34.0673,-118.2499
4751,34.0401,-118.2459
4752,34.0494,-118.2581
4753,34.0459,-118.2545
4754,34.0383,-118.254
4755,34.0412,-118.2436
4756,34.0502,-118.254
4757,34.037,-118.2583
4758,34.0423,-118.2666
4759,34.0442,-118.2439
4760,34.0472,-118.2385
4761,34.0481,-118.2507
4762,34.0581,-118.2371
4763,34.0474,-118.2496
4764,34.0589,-118.2527
4765,34.0442,-118.2439
4766,34.0442,-118.268
4767,34.0495,-118.2494
4768,34.0495,-118.2494
4769,34.0378,-118.2561
4770,34.0671,-118.2483
4771,34.0446,-118.245
4772,34.0446,-118.245
4773,34.0502,-118.254
4774,34.053,-118.2513
4775,34.0364,-118.2603
4776,34.0408,-118.2428
4777,34.0502,-118.2574
4778,34.0661,-118.2476
4779,34.0466,-118.259
4780,34.0487,-118.2588
4781,34.0488,-118.2518
4782,34.043,-118.2571
4783,34.071,-118.2302
4784,34.0495,-118.2494
4785,34.0438,-118.2547
4786,34.0376,-118.2594
4787,34.0607,-118.2494
4788,34.0407,-118.2468
4789,34.0502,-118.254
4790,34.0357,-118.2593
4791,34.0423,-118.2452


5115,34.0578,-118.252
5116,34.0474,-118.2496
5117,34.0396,-118.2451
5118,34.0474,-118.2496
5119,34.0493,-118.2418
5120,34.0337,-118.261
5121,34.045,-118.264
5122,34.0444,-118.2407
5123,34.0437,-118.2652
5124,34.0644,-118.2387
5125,34.0532,-118.2425
5126,34.0458,-118.243
5127,34.0473,-118.2566
5128,34.0419,-118.2485
5129,34.0342,-118.2636
5130,34.0346,-118.2323
5131,34.0544,-118.2466
5132,34.0449,-118.2458
5133,34.0444,-118.2628
5134,34.0525,-118.2357
5135,34.0416,-118.262
5136,34.0467,-118.2485
5137,34.0503,-118.2504
5138,34.0417,-118.2443
5139,34.0428,-118.2461
5140,34.0363,-118.2677
5141,34.0606,-118.2439
5142,34.0444,-118.2628
5143,34.0516,-118.2561
5144,34.0451,-118.2604
5145,34.043,-118.242
5146,34.0481,-118.2507
5147,34.0473,-118.2566
5148,34.0495,-118.2494
5149,34.0442,-118.2439
5150,34.0389,-118.2643
5151,34.048,-118.2577
5152,34.0336,-118.2626
5153,34.0481,-118.2542
5154,34.0416,-118.255
5155,34.0423,-118.2452
5156,34.0469,-118.2644
5157,34.0563,-118.2374
5158,34.0471,-118.249

In [185]:
results_df.head()

,:@computed_region_2dna_qi2s,:@computed_region_k96s_3jcv,:@computed_region_kqwf_mjcx,:@computed_region_qz3q_ghft,:@computed_region_tatf_ua23,:@computed_region_ur2y_g4cx,area_id,area_name,crm_cd,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_desc,cross_street,date_occ,date_rptd,dr_no,location,location_1,mocodes,premis_cd,premis_desc,rpt_dist_no,status,status_desc,time_occ,vict_age,vict_descent,vict_sex,weapon_desc,weapon_used_cd,lat,lon,zipcode,year
0,76,703,9,23075,1319,NaN,01,Central,310,310,NaN,NaN,BURGLARY,NaN,2010-08-06T00:00:00.000,2010-08-07T00:00:00.000,100101320,300 E 8TH ST,"{'type': 'Point', 'coordinates': [-118.2517, 3...",0324,203,OTHER BUSINESS,0176,IC,Invest Cont,1730,25,H,F,NaN,NaN,34.0413,-118.2517,90014,2010
1,76,542,9,23074,1502,NaN,01,Central,210,210,NaN,NaN,ROBBERY,LOS ANGELES,2010-11-25T00:00:00.000,2010-11-26T00:00:00.000,100101643,5TH,"{'type': 'Point', 'coordinates': [-118.2476, 3...",0411,102,SIDEWALK,0145,IC,Invest Cont,2100,51,H,M,OTHER KNIFE,207,34.0461,-118.2476,90014,2010
2,76,542,9,23074,1502,NaN,01,Central,946,946,NaN,NaN,OTHER MISCELLANEOUS CRIME,NaN,2010-01-27T00:00:00.000,2010-01-29T00:00:00.000,100105528,100 E 5TH ST,"{'type': 'Point', 'coordinates': [-118.2485, 3...",NaN,101,STREET,0154,IC,Invest Cont,1430,43,W,M,NaN,NaN,34.0467,-118.2485,90014,2010
3,26,626,7,23667,1086,NaN,07,Wilshire,930,930,NaN,NaN,CRIMINAL THREATS - NO WEAPON DISPLAYED,NaN,2010-03-12T00:00:00.000,2010-03-12T00:00:00.000,100700660,7600 BEVERLY BL,"{'type': 'Point', 'coordinates': [-118.3555, 3...",0421,109,PARK/PLAYGROUND,0724,AA,Adult Arrest,1050,41,B,M,VERBAL THREAT,511,34.0761,-118.3555,90036,2010
4,26,635,7,23667,332,NaN,07,Wilshire,626,626,NaN,NaN,INTIMATE PARTNER - SIMPLE ASSAULT,NaN,2010-03-13T00:00:00.000,2010-03-13T00:00:00.000,100700661,600 S DETROIT ST,"{'type': 'Point', 'coordinates': [-118.3451, 3...",0400 2000,502,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",0735,AA,Adult Arrest,0600,27,O,F,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",400,34.0649,-118.3451,90036,2010


In [172]:
# dates

In [174]:
# construct a basic timedelta and print it
print (timedelta(days=365, hours=0, minutes=0))
# print today's date
print ("today is: " + str(datetime.now()))
# print today's date one year from now
print ("one year from now it will be:" + str(datetime.now() - timedelta(days=365)))


365 days, 0:00:00
today is: 2019-03-22 12:52:39.459572
one year from now it will be:2018-03-22 12:52:39.460572


In [175]:
now = datetime.now()
print(now)
print(timedelta(days=365))
# 1year = datetime.now() - timedelta(days=365)
# 1year
print(type(now))

2019-03-22 12:52:39.479559
365 days, 0:00:00
<class 'datetime.datetime'>


In [186]:
# group by zipcode /crm_cd ; count number of crimes (dr_no) for year 2018
aggregation = { 
     'dr_no': 'nunique' 
}
df_1 = results_df.groupby(['zipcode','crm_cd']).agg(aggregation)
df_1

dr_no
zipcode crm_cd       
90002   230         1
90003   110         1
90005   236         1
        626         5
        946         1
90006   121         1
90007   230         1
        310         1
90012   940         1
90014   210         3
        310         1
        331         1
        946         1
90015   930         1
90016   121         1
        230         3
        624         1
        626         7
        888         1
        901         1
        930         6
90017   210         1
        343         1
90018   230         3
        236         1
        626         1
        930         2
90019   230        25
        236         6
        624         3
...               ...
90048   930         4
        946         2
90057   954         1
90069   230         4
        624         1
        930         1
90071   230         2
        251         1
        626         1
90077   354         1
90211   210         1
        230         1
        626         1
90501   812         2
90744   626         1
91303   649         1
91316   310         1
91331   110         1
91335   310         2
        888         1
91343   649         1
91356   310         1
91402   812         1
91405   510         2
91406   902         1
91411   510         1
        740         1
91423   649         1
91436   310         1
91607   649         1

[83 rows x 1 columns]

In [176]:
# rolling year

In [177]:
# 5 years

In [178]:
# income

In [179]:
# home value

In [180]:
result

SimpleZipcode(zipcode='90079', zipcode_type='Standard', major_city='Los Angeles', post_office_city='Los Angeles, CA', common_city_list=['Los Angeles'], county='Los Angeles County', state='CA', lat=34.053, lng=-118.245, timezone='Pacific', radius_in_miles=0.11363636363636363, area_code_list=['213', '323', '424', '626', '747', '818'], population=0, population_density=None, land_area_in_sqmi=0.0, water_area_in_sqmi=0.0, housing_units=0, occupied_housing_units=0, median_home_value=None, median_household_income=None, bounds_west=-118.256716, bounds_east=-118.254093, bounds_north=34.041599, bounds_south=34.039571)